Clean up

In [2]:
from datetime import datetime, timedelta
import time
import pandas as pd
import os

In [3]:
def cleanUp(df):
  #nullwaarden verwijderen
  df.dropna(inplace=True)

  #tijd aanpassen
  #tijdformaat ..:..:..
  tijd_regex = r'^\d{2}:\d{2}:\d{2}$'
  #omzetten naar datetime
  df['date'] = pd.to_datetime(df['dateDiff']).dt.date
  #filter rijen met formaat
  df = df[df['startTime'].str.match(tijd_regex, na=False) & df['rLength'].str.match(tijd_regex, na=False)].copy()
  #afleveringlengte naar seconden omzetten 
  df['Lengte_sec'] = pd.to_timedelta(df['rLength']).dt.total_seconds().astype(int)
  #kijkers naar int
  df['Kijkers'] = df['rateInK'].dropna().astype(str).str.replace('.', '', regex=False).astype(int)

  #uren met 24+
  def time_cor(rij):
    tijdArr = rij['startTime'].split(':')
    if int(tijdArr[0]) >= 24:
      tijdArr[0] = str(int(tijdArr[0]) - 24).zfill(2)
      rij['date'] += timedelta(days=1)
    rij['startTime'] = ':'.join(tijdArr)
    return rij
  
  df = df.apply(time_cor, axis=1)

  #1 kolom voor beide data
  df['FullDate'] = pd.to_datetime(df['date'].astype(str) 
                                  + " " + df['startTime'].astype(str))
  
  #hour en minute voor join later on
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute

  #kolommen verwijderen die niet nodig meer zijn
  df.drop(['startTime', 'rLength', 'rateInK', 'ranking', 'live'], axis=1, inplace=True)

  #de nieuwe dataframe
  df = df[['FullDate', 'date', 'hour','minute', 'channel', 'description', 'Lengte_sec', 'Kijkers']]

  #hernoemen kolommen
  df.rename(columns={'description':'Programma', 'channel':'Kanaal'}, inplace=True)

  return df

In [4]:
Kijkcijfers = pd.read_csv('./data csv/kijkcijfersdataraw.csv')
Kijkcijfers = cleanUp(Kijkcijfers)
Kijkcijfers.sample(10)

C:\Users\krist\AppData\Local\Temp\ipykernel_34648\78328727.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['startTime']).dt.hour
C:\Users\krist\AppData\Local\Temp\ipykernel_34648\78328727.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['minute'] = pd.to_datetime(df['startTime']).dt.minute


,FullDate,date,hour,minute,Kanaal,Programma,Lengte_sec,Kijkers
19684,2019-06-12 13:33:50,2019-06-12,13,33,EEN,WIELRENNEN. RONDE VAN BELGIE - RIT 1 SINT-NIKL...,7237,237746
32253,2021-03-04 18:59:57,2021-03-04,18,59,VTM,NIEUWS 19U VTM,3237,706558
62519,2025-04-07 20:14:35,2025-04-07,20,14,VRT 1,THUIS,1457,869193
13241,2018-07-26 21:24:26,2018-07-26,21,24,VTM,MET VIER IN BED,2206,290034
29325,2020-10-08 19:42:59,2020-10-08,19,42,VIER,HUIZENJAGERS,2437,226069
7035,2017-09-19 20:37:52,2017-09-19,20,37,VTM,THE WALL,3050,573326
44036,2022-09-25 22:26:42,2022-09-25,22,26,EEN,HET JOURNAAL LAAT,1042,176528
18752,2019-04-27 18:59:49,2019-04-27,18,59,VTM,NIEUWS 19U VTM,2453,442317
6857,2017-09-10 20:34:21,2017-09-10,20,34,VTM,DIE ANOTHER DAY,7375,338050
17233,2019-02-10 19:55:16,2019-02-10,19,55,VTM,DE KOTMADAM,1793,409286


Weerdata mergen

In [5]:
weerData = pd.read_csv('./data csv/weerdataraw.csv')
weerData['timestamp'] = pd.to_datetime(weerData['timestamp'])
#naar zelfde formaat als kijkcijfer datum
weerData['datetime'] = weerData['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

#hour voor join later on
weerData['hour'] = pd.to_datetime(weerData['datetime']).dt.hour
weerData['minute'] = pd.to_datetime(weerData['datetime']).dt.minute
weerData['date'] = pd.to_datetime(weerData['datetime']).dt.date

#verwijder kolom
weerData = weerData.drop(columns=['timestamp'])

weerData = weerData[['datetime', 'date' ,'hour', 'minute', 'temperature_2m', 'apparent_temperature', 
                           'rain', 'snowfall', 'weather_code', 'cloud_cover', 
                           'wind_speed_10m', 'sunshine_duration']]

#hernoemen kolommen
weerData.rename(columns={'temperature_2m':'Temperatuur', 'apparent_temperature':'Gevoelstemp', 'wind_speed_10m': 'Windsnelheid', 'rain':'Regen', 'snowfall': 'Sneeuw', 'weather_code':'Weercode', 'cloud_cover':'Bewolking', 'sunshine_duration':'Zonnenschijn'}, inplace=True)

weerData.sample(10)

,datetime,date,hour,minute,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn
51749,2022-08-27 05:00:00,2022-08-27,5,0,15.4,15.2,0.0,0.0,0.0,13.0,5.4,0.0
30279,2020-03-15 15:00:00,2020-03-15,15,0,11.7,7.5,0.0,0.0,3.0,100.0,21.2,0.0
4459,2017-04-04 19:00:00,2017-04-04,19,0,11.4,8.8,0.0,0.0,0.0,18.0,14.0,3600.0
62215,2023-11-06 07:00:00,2023-11-06,7,0,8.6,4.4,0.0,0.0,1.0,37.0,22.8,0.0
31436,2020-05-02 20:00:00,2020-05-02,20,0,12.4,10.0,0.0,0.0,1.0,23.0,8.4,3600.0
18478,2018-11-09 22:00:00,2018-11-09,22,0,10.6,7.3,0.0,0.0,3.0,100.0,20.7,0.0
9484,2017-10-31 04:00:00,2017-10-31,4,0,7.3,4.8,0.0,0.0,2.0,74.0,9.4,0.0
16228,2018-08-08 04:00:00,2018-08-08,4,0,20.4,20.4,0.0,0.0,2.0,50.0,15.7,0.0
62368,2023-11-12 16:00:00,2023-11-12,16,0,8.2,6.0,0.2,0.0,51.0,100.0,7.6,0.0
6987,2017-07-19 03:00:00,2017-07-19,3,0,21.0,19.5,0.0,0.0,3.0,85.0,19.1,0.0


Mergen weerdata en kijkcijferdata op FullDate

In [10]:
kijkcijfersWeer = pd.merge(Kijkcijfers, weerData, on=['date', 'hour'], how='left')

kijkcijfersWeer = kijkcijfersWeer[['FullDate', 'date', 'hour', 'Kanaal', 'Programma', 'Lengte_sec', 'Kijkers', 'Temperatuur', 'Gevoelstemp', 'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid', 'Zonnenschijn']]

kijkcijfersWeer.sample(10)

#kijkcijfersWeer.to_csv('./data csv/blablableble.csv')

,FullDate,date,hour,Kanaal,Programma,Lengte_sec,Kijkers,Temperatuur,Gevoelstemp,Regen,Sneeuw,Weercode,Bewolking,Windsnelheid,Zonnenschijn
49493,2023-08-08 21:11:03,2023-08-08,21,VTM,MET VIER IN BED,2455,566602,15.3,14.4,3.7,0.0,63.0,100.0,14.3,0.00
18126,2019-03-31 18:59:51,2019-03-31,18,VTM,NIEUWS 19U VTM,2782,452177,12.2,8.1,0.0,0.0,1.0,25.0,20.5,3600.00
41937,2022-07-20 17:44:35,2022-07-20,17,EEN,DE ZOO ACHTER DE SCHERMEN,1518,304674,25.0,24.8,0.0,0.0,3.0,100.0,13.7,3600.00
30631,2021-01-02 18:25:20,2021-01-02,18,VTM,BEESTIG,1650,207574,3.4,1.0,0.0,0.0,3.0,96.0,5.1,0.00
39377,2022-03-15 19:38:06,2022-03-15,19,PLAY4,BIG BROTHER,2502,225753,10.8,8.5,0.0,0.0,3.0,100.0,8.0,0.00
20748,2019-08-09 18:21:37,2019-08-09,18,VTM,DE KLINIEK,1521,210972,24.5,23.7,0.0,0.0,1.0,30.0,22.8,2223.24
957,2016-11-19 20:40:04,2016-11-19,20,Q2,THE INTERNATIONAL,6432,164924,5.7,1.3,0.0,0.0,3.0,100.0,17.7,0.00
14863,2018-10-19 18:59:53,2018-10-19,18,VTM,NIEUWS 19U VTM,3102,577440,14.6,12.9,0.0,0.0,2.0,54.0,10.2,1686.44
27914,2020-08-18 20:00:05,2020-08-18,20,Canvas,TER ZAKE,1960,197447,21.4,22.5,0.0,0.0,1.0,35.0,4.4,3600.00
48057,2023-05-27 18:39:29,2023-05-27,18,VRT 1,VLAANDEREN VAKANTIELAND,1092,226818,21.3,19.3,0.0,0.0,0.0,0.0,11.0,3600.00


debug box

In [9]:
# Bekijk de kolomnamen in Kijkcijfers en weerData
print(Kijkcijfers.columns)
print(weerData.columns)

#nullrows aanwezig?
print(f"null-rows: \n{kijkcijfersWeer.isnull().sum()}")



Index(['FullDate', 'date', 'hour', 'minute', 'Kanaal', 'Programma',
       'Lengte_sec', 'Kijkers'],
      dtype='object')
Index(['datetime', 'date', 'hour', 'minute', 'Temperatuur', 'Gevoelstemp',
       'Regen', 'Sneeuw', 'Weercode', 'Bewolking', 'Windsnelheid',
       'Zonnenschijn'],
      dtype='object')
null-rows: 
FullDate        0
date            0
hour            0
Kanaal          0
Programma       0
Lengte_sec      0
Kijkers         0
Temperatuur     0
Gevoelstemp     0
Regen           0
Sneeuw          0
Weercode        0
Bewolking       0
Windsnelheid    0
Zonnenschijn    0
dtype: int64


In [8]:
kijkcijfersWeer.dropna(inplace=True)
#csv aanmaken als het niet bestaat
csv_file_path = os.path.join('.','data csv','kijkcijfersWeerRaw.csv')
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
#data naar csv
kijkcijfersWeer.to_csv(csv_file_path, index=False)